In [0]:
import torch
from torchtext import data
import json
import spacy
from torch.utils.data import Dataset, DataLoader
import random
from tqdm import tqdm

print(torch.cuda.get_device_name(0))
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Tesla P100-PCIE-16GB


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
path = "/content/gdrive/My Drive/NLP-FinalProject/data15.csv"
# path = "/content/gdrive/My Drive/NLP-FinalProject/ss_dev_test.csv"

In [0]:
import pandas as pd
df = pd.read_csv(path)
df.head()

In [0]:
df.shape

In [0]:
df[df.label == True][['claim', 'sentence']].iloc[1]['sentence']

In [0]:
import spacy.lang.en

In [0]:
TEXT = data.Field(use_vocab=True, include_lengths=True, tokenize='spacy')
LABEL = data.LabelField()

In [0]:
data_fields = {'label': ('label', LABEL), 'sentence': ('sentence', TEXT), 'claim': ('claim', TEXT)}

In [0]:
train_data = data.TabularDataset(path, format = "CSV", fields=data_fields, skip_header=False)

In [0]:
train_data, valid_data = train_data.split(split_ratio = 0.9, random_state = random.seed(SEED))

In [0]:
print (vars(train_data.examples[0]))

{'label': 'False', 'sentence': ['Streptococcus', 'agalactiae', 'is', 'the', 'species', 'designation', 'for', 'streptococci', 'belonging', 'to', 'group', 'B', 'of', 'the', 'Lancefield', 'classification', '.'], 'claim': ['Streptococcus', 'belongs', 'to', 'a', 'class', '.']}


In [0]:
TEXT.build_vocab(train_data, 
                #  vectors = "charngram.100d",
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [06:31, 2.20MB/s]                           
100%|█████████▉| 399852/400000 [00:15<00:00, 26728.28it/s]

In [0]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 201853
Unique tokens in LABEL vocabulary: 2


In [0]:
print(TEXT.vocab.freqs.most_common(20))

[('.', 3811329), (',', 3193635), ('the', 2978927), ('of', 1702859), ('in', 1681871), ('and', 1587247), ('a', 1523453), ('is', 1165789), ('was', 888493), ('The', 824598), ('to', 734394), ('-RRB-', 600193), ('-LRB-', 600096), ('-', 599632), ('by', 556858), ('`', 526149), ('for', 495902), ('on', 471054), ('as', 456789), ("'s", 449574)]


In [0]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', '.', ',', 'the', 'of', 'in', 'and', 'a', 'is']


In [0]:
print (vars(LABEL.vocab))

{'freqs': Counter({'False': 1816212, 'True': 181970}), 'itos': ['False', 'True'], 'stoi': defaultdict(<function _default_unk_index at 0x7fd72f9fcae8>, {'False': 0, 'True': 1}), 'vectors': None}


In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("We are working with ", device)

train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    sort_key = lambda x: (len(x.claim)),
    device = device)

We are working with  cuda


In [0]:
# for batch in train_iterator:
#     print (batch.claim[1])
#     print (batch.sentence[1])
#     break

In [0]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.n_hidden = hidden_dim
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=n_layers)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    
    def forward_once(self, text, text_lengths):
        embed = self.dropout(self.embedding(text))
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embed, text_lengths, enforce_sorted=False)
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        hidden = hidden[-1, :, :]
        hidden = self.dropout(hidden)
        return hidden

    def forward(self, claim, sentence):
        #text = [sent_len, batch_size]
        s1 = self.forward_once(claim[0], claim[1])
        s2 = self.forward_once(sentence[0], sentence[1])
        # print (s1.shape, s2.shape)
        s_new = torch.cat((s1, s2), dim= 1)
        return self.fc(s_new)

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = len(LABEL.vocab)
N_LAYERS = 1
# N_LAYERS = 1
BIDIRECTIONAL = False
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT,
            PAD_IDX)

In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 20,552,918 trainable parameters


In [0]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([201853, 100])


In [0]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.4374, -0.6741, -0.0336,  ...,  1.0537,  0.5680,  0.1552],
        [-0.9062, -0.1182, -0.3180,  ..., -0.3090, -0.3151,  2.0463],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [-0.2099,  0.2971, -0.0353,  ...,  0.0715, -0.1213,  0.0088],
        [-1.1329,  0.5331, -1.0437,  ...,  1.3489, -0.5409, -1.0452],
        [-1.2625,  1.1972,  0.7535,  ...,  0.6080, -0.0259,  0.9606]])

In [0]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [-0.2099,  0.2971, -0.0353,  ...,  0.0715, -0.1213,  0.0088],
        [-1.1329,  0.5331, -1.0437,  ...,  1.3489, -0.5409, -1.0452],
        [-1.2625,  1.1972,  0.7535,  ...,  0.6080, -0.0259,  0.9606]])


In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [0]:
criterion = nn.CrossEntropyLoss(reduction='mean')

model = model.to(device)
criterion = criterion.to(device)

In [0]:
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score

In [0]:
# def binary_accuracy(preds, y):
#     """
#     Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
#     """
#     #round predictions to the closest integer
#     rounded_preds = torch.sigmoid(preds)
#     # correct = (rounded_preds == y).float() #convert into float for division 
#     # acc = correct.sum() / len(correct)
#     print (preds, rounded_preds, y)
#     print(classification_report(rounded_preds.cpu().detach().numpy(), y.cpu()))
#     # return acc
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    metric = {'0': {}, '1': {}}
    f1 = classification_report(max_preds.cpu().detach().numpy(), y.cpu(), output_dict=True)
    try:
            
        metric['0']['pre'] = f1.get('0').get('precision', 0)
        metric['0']['recall'] = f1.get('0').get('recall', 0)
        metric['0']['f1'] = f1.get('0').get('f1', 0)
        if '1' in f1.keys():
            metric['1']['pre'] = f1.get('1').get('precision', 0)
            metric['1']['recall'] = f1.get('1').get('recall', 0)
            metric['1']['f1'] = f1.get('1').get('f1', 0)
        else:
            metric['1']['pre'] = 0
            metric['1']['recall'] = 0
            metric['1']['f1'] = 0
            
    except Exception as e:
        print (e)
        import pdb; pdb.set_trace()

    f1_total = f1_score(max_preds.cpu().detach().numpy(), y.cpu())
    pre = precision_score(max_preds.cpu().detach().numpy(), y.cpu(), average='macro')
    rec = recall_score(max_preds.cpu().detach().numpy(), y.cpu(), average='macro')
    return correct.sum() / torch.FloatTensor([y.shape[0]]), metric, f1_total, pre, rec

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    metrics = {'0': {'recall': 0, 'pre': 0, 'f1': 0}, '1': {'recall': 0, 'pre': 0, 'f1': 0}}
    epoch_f1_total = 0
    epoch_pre = 0
    epoch_rec = 0
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        # try:
        predictions = model(batch.claim, batch.sentence).squeeze(1)
        # except Exception as e:
        #     print (batch.claim[1], batch.sentence[1])
        # print ("Predictions shape: ", predictions.shape)
        loss = criterion(predictions, batch.label)
        
        acc, metric, f1_total, pre, rec = categorical_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        for key in metrics.keys():
            for m in metrics[key].keys():
                metrics[key][m] += metric[key][m]
        epoch_f1_total += f1_total
        epoch_pre += pre
        epoch_rec += rec
    
    for key in metrics.keys():
        for m in metrics[key].keys():
            metrics[key][m] = metrics[key][m]/len(iterator)
        

    return epoch_loss / len(iterator), epoch_acc / len(iterator), metrics, epoch_f1_total/ len(iterator), epoch_pre/ len(iterator), epoch_rec/ len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    metrics = {'0': {'recall': 0, 'pre': 0, 'f1': 0}, '1': {'recall': 0, 'pre': 0, 'f1': 0}}
    epoch_f1_total = 0
    epoch_pre = 0
    epoch_rec = 0
    model.eval()
    
    with torch.no_grad():
        for batch in tqdm(iterator):
        
            predictions = model(batch.claim, batch.sentence).squeeze(1)
            
            acc, metric, f1_total, pre, rec = categorical_accuracy(predictions, batch.label) 
            loss = criterion(predictions, batch.label)       

            epoch_acc += acc.item()
            epoch_loss += loss.item()

            for key in metrics.keys():
                for m in metrics[key].keys():
                    metrics[key][m] += metric[key][m]
            epoch_f1_total += f1_total
            epoch_pre += pre
            epoch_rec += rec
        
        for key in metrics.keys():
            for m in metrics[key].keys():
                metrics[key][m] = metrics[key][m]/len(iterator)
            

    return epoch_loss/ len(iterator), epoch_acc / len(iterator), metrics, epoch_f1_total/ len(iterator), epoch_pre/ len(iterator), epoch_rec/ len(iterator)

In [0]:
N_EPOCHS = 5
path = F"/content/gdrive/My Drive/NLP-FinalProject/sentence_selection1.pt"
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc, metrics, train_f1_total, total_pre, total_rec = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc, v_metrics, _, _, _ = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    # if valid_loss < best_valid_loss:
    #     best_valid_loss = valid_loss
    torch.save(model.state_dict(), path)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% |Train Pre: {total_pre*100:.2f}% |Train rec: {total_rec*100:.2f}% | Train F1 Total: {train_f1_total:.2f} score')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    print (f"\t Train metrics: {metrics}")
    print (f"\t Val metrics: {v_metrics}")

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: 

Epoch: 01 | Epoch Time: 14m 0s
	Train Loss: 0.227 | Train Acc: 91.65% |Train Pre: 59.11% |Train rec: 69.78% | Train F1 Total: 0.25 score
	 Val. Loss: 0.197 |  Val. Acc: 92.51%
	 Train metrics: {'0': {'recall': 0.9250655670095795, 'pre': 0.9879994421519148, 'f1': 0.0}, '1': {'recall': 0.4677471786393307, 'pre': 0.19148207856379518, 'f1': 0.0}}
	 Val metrics: {'0': {'recall': 0.9376814490446154, 'pre': 0.9824460353604567, 'f1': 0.0}, '1': {'recall': 0.6179224929729259, 'pre': 0.3354114545396972, 'f1': 0.0}}



100%|██████████| 3470/3470 [00:40<00:00, 85.05it/s]


Epoch: 02 | Epoch Time: 13m 54s
	Train Loss: 0.196 | Train Acc: 92.49% |Train Pre: 65.60% |Train rec: 78.10% | Train F1 Total: 0.40 score
	 Val. Loss: 0.184 |  Val. Acc: 92.95%
	 Train metrics: {'0': {'recall': 0.9365407118309267, 'pre': 0.9836891378114698, 'f1': 0.0}, '1': {'recall': 0.6233611620374023, 'pre': 0.3261918976447384, 'f1': 0.0}}
	 Val metrics: {'0': {'recall': 0.9471785759570271, 'pre': 0.976588511853261, 'f1': 0.0}, '1': {'recall': 0.6448506528693844, 'pre': 0.452215448256514, 'f1': 0.0}}



100%|██████████| 3470/3470 [00:41<00:00, 83.15it/s]


Epoch: 03 | Epoch Time: 13m 57s
	Train Loss: 0.186 | Train Acc: 92.84% |Train Pre: 68.03% |Train rec: 79.93% | Train F1 Total: 0.45 score
	 Val. Loss: 0.180 |  Val. Acc: 93.17%
	 Train metrics: {'0': {'recall': 0.9407436200475122, 'pre': 0.9828083849856654, 'f1': 0.0}, '1': {'recall': 0.6557223716005877, 'pre': 0.3757430363622511, 'f1': 0.0}}
	 Val metrics: {'0': {'recall': 0.9416863510768003, 'pre': 0.9856670377453749, 'f1': 0.0}, '1': {'recall': 0.694334844630233, 'pre': 0.39036044766952616, 'f1': 0.0}}



100%|██████████| 3470/3470 [00:41<00:00, 84.34it/s]


Epoch: 04 | Epoch Time: 14m 2s
	Train Loss: 0.181 | Train Acc: 93.04% |Train Pre: 69.20% |Train rec: 80.86% | Train F1 Total: 0.47 score
	 Val. Loss: 0.175 |  Val. Acc: 93.29%
	 Train metrics: {'0': {'recall': 0.9428391792607506, 'pre': 0.9827166803674371, 'f1': 0.0}, '1': {'recall': 0.672572996100463, 'pre': 0.3994216715809163, 'f1': 0.0}}
	 Val metrics: {'0': {'recall': 0.9441590014182509, 'pre': 0.9841451858089673, 'f1': 0.0}, '1': {'recall': 0.693877993604218, 'pre': 0.4173048661233101, 'f1': 0.0}}



100%|██████████| 3470/3470 [00:41<00:00, 84.26it/s]


Epoch: 05 | Epoch Time: 14m 0s
	Train Loss: 0.178 | Train Acc: 93.16% |Train Pre: 69.93% |Train rec: 81.23% | Train F1 Total: 0.49 score
	 Val. Loss: 0.174 |  Val. Acc: 93.35%
	 Train metrics: {'0': {'recall': 0.9441341542815183, 'pre': 0.9826010466928295, 'f1': 0.0}, '1': {'recall': 0.6784055446969272, 'pre': 0.4139342264287856, 'f1': 0.0}}
	 Val metrics: {'0': {'recall': 0.94749640321431, 'pre': 0.9808642465873141, 'f1': 0.0}, '1': {'recall': 0.683738694062902, 'pre': 0.4566591265762721, 'f1': 0.0}}


In [0]:
path = "/content/gdrive/My Drive/NLP-FinalProject/ss_dev_test.csv"

In [0]:
TEXT = data.Field(use_vocab=True, include_lengths=True, tokenize='spacy')
LABEL = data.LabelField()
data_fields = {'label': ('label', LABEL), 'sentence': ('sentence', TEXT), 'claim': ('claim', TEXT)}

In [0]:
test_data = data.TabularDataset(path, format = "CSV", fields=data_fields, skip_header=False)

In [0]:
TEXT.build_vocab(test_data, 
                #  vectors = "charngram.100d",
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(test_data)

.vector_cache/glove.6B.zip: 862MB [06:30, 2.21MB/s]                          
100%|█████████▉| 398444/400000 [00:15<00:00, 25186.90it/s]

In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("We are working with ", device)

test_iterator = data.BucketIterator(
    test_data, 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    sort_key = lambda x: (len(x.claim)),
    device = device)

We are working with  cuda


In [0]:
INPUT_DIM = 201853
# len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 2
# len(LABEL.vocab)
N_LAYERS = 1
# N_LAYERS = 1
BIDIRECTIONAL = False
DROPOUT = 0.5
PAD_IDX = 0
# TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT,
            PAD_IDX)

In [0]:
criterion = nn.CrossEntropyLoss(reduction='mean')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("We are working with ", device)

path = F"/content/gdrive/My Drive/NLP-FinalProject/sentence_selection1.pt"
model.load_state_dict(torch.load(path, map_location=device))
# test_loss, test_acc = evaluate(model, test_iterator, criterion)

# print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')
model = model.to(device)
# criterion = criterion.to(device)

We are working with  cuda


In [0]:
import spacy
nlp = spacy.load('en')

def tokenizer(sent):
    return torch.LongTensor([TEXT.vocab.stoi[tok.text] for tok in nlp.tokenizer(sent)]).unsqueeze(1).to(device)

def length(sent):
    return torch.LongTensor([sent.size()[0]])

def predict_sentiment(model, claim, sentence):
    model.eval()
    claim_tokenized = tokenizer(claim)
    claim_length = length(claim_tokenized)

    sentence_tokenized = tokenizer(sentence)
    sentence_length = length(sentence_tokenized)

    prediction = torch.sigmoid(model((claim_tokenized, claim_length), (sentence_tokenized, sentence_length)))
    if prediction.argmax(dim=1)[0]:
        return True
    else:
        return False

In [0]:
from tqdm import tqdm
start_time = time.time()

test_loss, test_acc, metrics, test_f1_total, total_pre, total_rec = evaluate(model, test_iterator, criterion)
# valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

end_time = time.time()

epoch_mins, epoch_secs = epoch_time(start_time, end_time)

# if valid_loss < best_valid_loss:
#     best_valid_loss = valid_loss
# torch.save(model.state_dict(), path)

print(f'\tTest Acc: {test_acc*100:.2f}% |Test Pre: {total_pre*100:.2f}% |Test rec: {total_rec*100:.2f}% | Test F1 Total: {train_f1_total:.2f} score')
print(f'\t Metrics: {metrics}')
# print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')


  0%|          | 1/10057 [00:01<3:28:28,  1.24s/it]/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

  0%|          | 11/10057 [00:01<2:26:20,  1.14it/s]/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

NameError: ignored

In [0]:
metrics

{'0': {'f1': 0.0, 'pre': 0.9707893616529578, 'recall': 0.9661785050537772},
 '1': {'f1': 0.0, 'pre': 0.036933250314477216, 'recall': 0.0381348930145787}}

In [0]:
# # test = df[df.label == True]
# from tqdm import tqdm
# predictions= []
# for value in tqdm(df.itertuples()):
#     predictions.append(predict_sentiment(model, value[2], value[5]))

Streaming output truncated to the last 5000 lines.
445804it [19:21, 372.11it/s]
445842it [19:21, 364.38it/s]
445881it [19:21, 370.82it/s]
445925it [19:22, 388.10it/s]
445968it [19:22, 399.29it/s]
446010it [19:22, 404.78it/s]
446051it [19:22, 402.45it/s]
446093it [19:22, 404.83it/s]
446134it [19:22, 387.87it/s]
446174it [19:22, 376.53it/s]
446212it [19:22, 376.21it/s]
446250it [19:22, 363.45it/s]
446292it [19:23, 377.14it/s]
446333it [19:23, 384.12it/s]
446374it [19:23, 388.42it/s]
446414it [19:23, 384.37it/s]
446453it [19:23, 372.34it/s]
446491it [19:23, 358.18it/s]
446528it [19:23, 353.28it/s]
446567it [19:23, 362.72it/s]
446605it [19:23, 365.38it/s]
446643it [19:23, 368.50it/s]
446683it [19:24, 377.09it/s]
446725it [19:24, 388.57it/s]
446765it [19:24, 386.39it/s]
446805it [19:24, 389.07it/s]
446844it [19:24, 377.69it/s]
446888it [19:24, 392.75it/s]
446928it [19:24, 392.38it/s]
446968it [19:24, 389.98it/s]
447008it [19:24, 381.07it/s]
447049it [19:25, 387.24it/s]
447095it [19:25, 404.

In [0]:
len(predictions)

643617

In [0]:
df.head()

,label,sentence,id,verifiable,claim,h_links,claim_label,article_title
0,False,Colin Rand Kaepernick -LRB- -LSB- ` kæpərnɪk -...,91198,NOT VERIFIABLE,Colin Kaepernick became a starting quarterback...,"['American football', 'American football', 'qu...",NOT ENOUGH INFO,['Colin Kaepernick']
1,False,Kaepernick played collegiate football at the U...,91198,NOT VERIFIABLE,Colin Kaepernick became a starting quarterback...,"['Nevada', 'Nevada Wolf Pack football', 'colle...",NOT ENOUGH INFO,['Colin Kaepernick']
2,False,Kaepernick was selected by the San Francisco 4...,91198,NOT VERIFIABLE,Colin Kaepernick became a starting quarterback...,"['San Francisco 49ers', 'San Francisco 49ers',...",NOT ENOUGH INFO,['Colin Kaepernick']
3,False,Kaepernick began his professional career as a ...,91198,NOT VERIFIABLE,Colin Kaepernick became a starting quarterback...,"['Alex Smith', 'Alex Smith', '2012 season', '2...",NOT ENOUGH INFO,['Colin Kaepernick']
4,False,He remained the team 's starting quarterback f...,91198,NOT VERIFIABLE,Colin Kaepernick became a starting quarterback...,"['quarterback', 'quarterback', 'Super Bowl', '...",NOT ENOUGH INFO,['Colin Kaepernick']


In [0]:
claim = 'Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.'
sentence = 'The Fox Broadcasting Company -LRB- often shortened to Fox and stylized as FOX -RRB- is an American English language commercial broadcast television network that is owned by the Fox Entertainment Group subsidiary of 21st Century Fox .'
predict_sentiment(model, claim, sentence)

(False, tensor([[0.8583, 0.2021]], device='cuda:0', grad_fn=<SigmoidBackward>))

In [0]:
class SentenceDataset:
    def __init__(self, file_name):
        self.file_name = file_name

    def __getitem__(self, idx):
        with open(self.file_name, 'r') as f:
            data = json.load(f)[idx]
        print (data.keys())
        out = dict()
        out['claim'] = data['claim']
        out['sentence'] = data['sentence']
        out['label'] = data['label']
        return out

In [0]:
path = "/content/gdrive/My Drive/NLP-FinalProject/ss_training_data1.json"
dataset= SentenceDataset(path)

In [0]:
dataset[0]

dict_keys(['sentence', 'claim', 'h_links', 'label', 'claim_label', 'article_title'])


{'claim': 'Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.',
 'label': False,
 'sentence': 'Nikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ̯ kʰʌsd̥ɐ ˈʋald̥ɑʊ̯ -RSB- ; born 27 July 1970 -RRB- is a Danish actor , producer and screenwriter .'}